If using colab, install necessary libraries

In [ ]:
! pip install urlextract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import requests     # 2.18.4
import json         # 2.0.9
import pandas as pd # 0.23.0
import numpy as np
import ast
import time
from urlextract import URLExtract as extract
extractor = extract()

ModuleNotFoundError: ignored

In [ ]:
# BFE Abnahme
packages = 'https://ckan.ogdch-abnahme.clients.liip.ch/api/3/action/package_search?fq=organization:(bundesamt-fur-energie-bfe)&rows=500'

# Make the HTTP request
response = requests.get(packages)

# Use the json module to load CKAN's response into a dictionary
response_dict = json.loads(response.content)

# Check the contents of the response
assert response_dict['success'] is True  # make sure if response is OK

# Get a list of all publications and their information (each publication is a dictionary)
data = response_dict['result']['results'].copy()

# get all upper keys
allKeys = []

# store information about each dataset in a frame
df = pd.DataFrame(data)[['id','title','url', 'relations', 'resources']]
df

,id,title,url,relations,resources
0,6cf5d98c-e259-4c40-a54b-a93b539eb79b,{'fr': 'Réseaux thermiques: offre de chaleur o...,https://www.bfe.admin.ch/waerme-und-kaelteangebot,[{'url': 'https://www.geocat.ch/geonetwork/srv...,"[{'protocol': 'WWW:DOWNLOAD:Text (.csv), comma..."
1,70faeac6-d660-492d-bc5b-dfdfa707007e,{'fr': 'Réseaux thermiques: demande concernant...,https://www.bfe.admin.ch/waerme-kaeltenachfrag...,[{'url': 'https://www.geocat.ch/geonetwork/srv...,"[{'protocol': 'WWW:DOWNLOAD:Geopackage (ogc)',..."
2,0a6ef29d-86b6-416f-99c3-5e6d26f1f506,{'fr': 'Situation et domaines attenants confor...,https://www.bfe.admin.ch/konsultationsbereiche...,[{'url': 'https://www.geocat.ch/geonetwork/srv...,"[{'protocol': 'OGC:WMS', 'cache_last_updated':..."
3,46ee8db4-c4e9-436a-ad39-bb67102bffec,{'fr': 'Plan sectoriel des lignes de transport...,https://www.bfe.admin.ch/sp-uebertragungsleitu...,[{'url': 'https://www.geocat.ch/geonetwork/srv...,"[{'protocol': 'WWW:DOWNLOAD:Geopackage (ogc)',..."
4,bb13b62d-6cd1-4602-acc1-bbe2aba197b0,{'fr': 'Réseaux thermiques: demande concernant...,https://www.bfe.admin.ch/waerme-kaeltenachfrag...,[{'url': 'https://www.geocat.ch/geonetwork/srv...,"[{'protocol': 'WWW:DOWNLOAD:Geopackage (ogc)',..."
...,...,...,...,...,...
100,a37cb9ae-50f9-4fa5-a48c-95e2c5f0625b,{'fr': 'Aptitude des toitures à utiliser l’éne...,https://www.bfe.admin.ch/bfe/de/home/versorgun...,{},"[{'byte_size': 0, 'cache_last_updated': None, ..."
101,979f37ed-fcb5-4400-802b-be0bb2833ea4,{'fr': 'Statistiques heddomadaires: production...,http://www.bfe.admin.ch/ogd51,[{'url': 'https://www.bfe.admin.ch/bfe/de/home...,"[{'byte_size': 0, 'cache_last_updated': None, ..."
102,73d41982-f474-409a-a129-418678921128,{'fr': 'Etude sur l'électricité hivernale d'or...,https://www.bfe.admin.ch/bfe/de/home/politik/e...,[{'url': 'https://www.bfe.admin.ch/bfe/de/home...,"[{'byte_size': 5819680686, 'cache_last_updated..."
103,e839fa78-ba22-4772-be16-0843b99b7161,{'fr': 'Résultats de la modélisation PE2050+ :...,https://www.bfe.admin.ch/bfe/de/home/politik/e...,[{'url': 'https://www.uvek-gis.admin.ch/BFE/st...,"[{'byte_size': 0, 'cache_last_updated': None, ..."


---------------------------------------------------------------------------

Functions

In [ ]:
def check_url(url):
    try:
        time.sleep(1)
        # head or get --> head is not working consistantly
        
        # redirect not working on every machine and not everytime... but this link technically works
        if url == 'https://bfe.admin.ch/shared-mobility-angebote':
            return True
        
        get = requests.get(url)
        
        if get.status_code < 400:
            return True
        else:
            return False
        
    except requests.exceptions.RequestException as e:
        print(e)
        raise SystemExit(f"{url}: is NOT reachable \nErr: {e}")

In [ ]:
def addHTTPS(url):
    adjusted = url.replace("http:", "https:" )
    adjusted = adjusted.replace("www", "https://www" )
    
    return adjusted.replace("https://https://www", "https://www" )

In [ ]:
def addHTTP(url):
    adjusted = url.replace("www", "http://www")
    return adjusted.replace("http://http://www", "http://www")

In [ ]:
def getURLs(original_string, urls):
    
    links = []

    badLinks       = []

    adjustedString = '' 
    for s in original_string:
      adjustedString = adjustedString + s


    needsToBeListed = False

    
    for url in urls:
      
      url_modified    = ''
      # only store links which are faulty or need a protocol update (https)
      
      
      # if https already used
      if 'https' in url:
        working = check_url(url)
        
        if not working:
          badLinks.append([url, '-'])
          #needsToBeListed = True
          
      # if https not in link
      else:
        
        url_modified = addHTTPS(url)
        working = check_url(url_modified)
        
        if working:
          needsToBeListed = True
          # replace in adjusted_string
          adjustedString = adjustedString.replace(url, url_modified)
          
        else:
          # does not change http: urls
          url_modified = addHTTP(url)
          working = check_url(url)
          
          if not working:
            badLinks.append([url, '-'])
                    
        
    if needsToBeListed:
      links.append([original_string, adjustedString])

    if len(badLinks) != 0:
      links = links + badLinks

    return links

In [ ]:
def checkString(string):
  
  urls = extractor.find_urls(string)
  
  # first remove elements which are not actual links
  urls = [url for url in urls if ('http' in url) or ('www' in url)]

  # if the same url appears multiple times
  urls = list(set(urls))
  
  return getURLs(string, urls)

In [ ]:
urls = ['http://www.google.ch', 'https://www.google.ch', 'www.google.ch', 'https://data.geo.admin.ch/ch.bfe.windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150_2056.fgdb.zip']
test = getURLs(str(urls), urls)

In [ ]:
for i in test:
  print(i)

["['http://www.google.ch', 'https://www.google.ch', 'www.google.ch', 'https://data.geo.admin.ch/ch.bfe.windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150_2056.fgdb.zip']", "['https://https://www.google.ch', 'https://https://www.google.ch', 'https://www.google.ch', 'https://data.geo.admin.ch/ch.bfe.windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150_2056.fgdb.zip']"]
['https://data.geo.admin.ch/ch.bfe.windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150_2056.fgdb.zip', '-']


In [ ]:
check_url('https://www.bfe.admin.ch/waerme-kaeltenachfrage-wohnen-und-dl')

True

------------------------------------------------------------------------------------------------------------------------------------------------------

For _url_ and _relations_

In [ ]:
cols = ['url', 'relations']

completeList = []

for index, row in df.iterrows():
    
    print(index)
    dataset_name = row['title']['de']
    id           = row['id']
    
    for col in cols:
  
        att_type = 'dataset'
        att      = col
        val      = row[col]

        liste = []

        if col == 'resources':
            
            att_type = 'resources'
            #change att
            print('nichts')


        else:
            if type(val) == str:
                liste = checkString(val)
            else:
                liste = checkString(str(val))


        if len(liste) == 0:
            continue
        else:
            for elem in liste:
                completeList.append([dataset_name, att_type, att, id, elem[0], elem[1]])


0
['https://www.bfe.admin.ch/waerme-und-kaelteangebot']
['https://www.geocat.ch/geonetwork/srv/ger/catalog.search#/metadata/b6c8d9c6-a2ca-435a-af0a-6a8ac94199fc']
1
['https://www.bfe.admin.ch/waerme-kaeltenachfrage-wohnen-und-dl']


KeyboardInterrupt: ignored

In [ ]:
f = pd.DataFrame(completeList)
f.columns=['title', 'type', 'attribute', 'id', 'old_link', 'new_link']
f

In [ ]:
f.to_csv('urlAndRelations.csv', index=False)

-------------------------------------------------------------------------------------------------------------------------------------------------------------

For _resources_

In [ ]:
# import module
from urllib.request import urlopen
from urllib.error import *
 
# try block to read URL

def checkLink(link):

  try:
      html = urlopen(link)
      
  # except block to catch
  # exception
  # and identify error
  except HTTPError as e:
      print("HTTP error", e)
      print(link)
      print()
      return False
      
  except URLError as e:
      print("Opps ! Page not found!", e)
      print(link)
      print()
      return False
  
  else:
      return True

In [ ]:
def getURLs(original_string, urls):

    print(urls)
    
    links = []

    badLinks       = []

    adjustedString = '' 
    for s in original_string:
      adjustedString = adjustedString + s


    needsToBeListed = False

    
    for url in urls:
      
      url_modified    = ''
      # only store links which are faulty or need a protocol update (https)
      
      
      # if https already used
      if 'https' in url:
        working = checkLink(url)
        
        if not working:
          badLinks.append([url, '-'])
          #needsToBeListed = True
          
      # if https not in link
      else:
        
        url_modified = addHTTPS(url)
        working = checkLink(url_modified)
        
        if working:
          needsToBeListed = True
          # replace in adjusted_string
          adjustedString = adjustedString.replace(url, url_modified)
          
        else:
          # does not change http: urls
          url_modified = addHTTP(url)
          working = checkLink(url)
          
          if not working:
            badLinks.append([url, '-'])
                    
        
    if needsToBeListed:
      links.append([original_string, adjustedString])

    if len(badLinks) != 0:
      links = links + badLinks


    

    return links

In [ ]:
def checkDictionary(l):

  resourceList = []
  
  for d in l:
    
    id = d['id']

    for key in d.keys():

      #print(d)

      value    = d[key]
      isString = True

      if type(value) != str:
        isString = False
        value    = str(d[key])


      urls = extractor.find_urls(value)
  
      # first remove elements which are not actual links
      urls = [url for url in urls if ('http' in url) or ('www' in url)]

      whatToChange = []

      if len(urls) != 0:
        whatToChange = getURLs(value, urls)
      
      for elem in whatToChange:
        resourceList.append([key] + [id] + elem)

  return resourceList

In [ ]:
d = df.iloc[40]['resources'].copy()
res = checkDictionary(d)

[{'byte_size': 0, 'protocol': 'OGC:WMS', 'cache_last_updated': None, 'issued': '2016-02-25T00:00:00', 'package_id': 'a7e83e99-3ee9-4ffc-b1d5-8a792537a161', 'datastore_contains_all_records_of_source_file': False, 'datastore_active': False, 'id': 'fbefbaa8-fcd7-4030-abdc-a38641febbf5', 'size': None, 'display_name': {'fr': 'ch.bfe.windenergie-geschwindigkeit_h100', 'de': 'ch.bfe.windenergie-geschwindigkeit_h100', 'en': 'ch.bfe.windenergie-geschwindigkeit_h100', 'it': 'ch.bfe.windenergie-geschwindigkeit_h100'}, 'title': {'fr': 'ch.bfe.windenergie-geschwindigkeit_h100', 'de': 'ch.bfe.windenergie-geschwindigkeit_h100', 'en': 'ch.bfe.windenergie-geschwindigkeit_h100', 'it': 'ch.bfe.windenergie-geschwindigkeit_h100'}, 'state': 'active', 'hash': '', 'description': {'fr': 'Vitesse du vent à 100 m du sol', 'de': 'Windgeschwindigkeit 100 m Höhe', 'en': 'Wind speed 100 metres above ground', 'it': 'Velocità vento a 100 m dal suolo'}, 'format': 'WMS', 'mimetype_inner': None, 'url_type': None, 'mimety

In [ ]:
for r in res:
  print(r)

['download_url', 'f0904fec-5bb3-4189-bf2f-4d7bd4a560ab', 'https://data.geo.admin.ch/ch.bfe.windenergie-geschwindigkeit_h100/windenergie-geschwindigkeit_h100/windenergie-geschwindigkeit_h100_2056.fgdb.zip', '-']
['url', 'f0904fec-5bb3-4189-bf2f-4d7bd4a560ab', 'https://data.geo.admin.ch/ch.bfe.windenergie-geschwindigkeit_h100/windenergie-geschwindigkeit_h100/windenergie-geschwindigkeit_h100_2056.fgdb.zip', '-']


In [ ]:
completeList = []

for index, row in df.iterrows():
  
  print(index)
  dataset_name = row['title']['de']
    
  att_type = 'resources'
  val      = row[att_type]

  liste = checkDictionary(val)
    
  if len(liste) != 0:
    for elem in liste:
      completeList.append([dataset_name] + [att_type] + elem)


0
Dictionary:
{'protocol': 'WWW:DOWNLOAD:Text (.csv), comma separated', 'cache_last_updated': None, 'issued': '2018-10-03T00:00:00', 'package_id': '6cf5d98c-e259-4c40-a54b-a93b539eb79b', 'datastore_contains_all_records_of_source_file': False, 'datastore_active': False, 'id': '02adbe17-bb43-4afc-b6a0-fb423fdcc6ee', 'size': None, 'display_name': {'fr': '', 'de': 'Csv', 'en': '', 'it': ''}, 'title': {'fr': '', 'de': 'Csv', 'en': '', 'it': ''}, 'download_url': 'https://data.geo.admin.ch/ch.bfe.fernwaerme-angebot/fernwaerme-angebot/fernwaerme-angebot_2056.csv.zip', 'created': '2023-03-02T04:02:20.263833', 'state': 'active', 'media_type': 'Text (.csv), comma separated', 'hash': '', 'description': {'fr': '', 'de': 'Csv', 'en': '', 'it': ''}, 'format': 'CSV', 'mimetype_inner': None, 'url_type': None, 'mimetype': None, 'cache_url': None, 'name': {'fr': '', 'de': 'Csv', 'en': '', 'it': ''}, 'language': ['de'], 'rights': 'NonCommercialAllowed-CommercialAllowed-ReferenceRequired', 'url': 'https://

In [ ]:
completeList

[['Aktuelle Pilot- und Demonstrationsprojekte des Bundesamts für Energie',
  'resources',
  'download_url',
  'ff6a28fe-f691-4ee1-87df-50ef4d8decf6',
  'https://data.geo.admin.ch/ch.bfe.energieforschung/gpkg/2056/ch.bfe.energieforschung.zip',
  '-'],
 ['Aktuelle Pilot- und Demonstrationsprojekte des Bundesamts für Energie',
  'resources',
  'url',
  'ff6a28fe-f691-4ee1-87df-50ef4d8decf6',
  'https://data.geo.admin.ch/ch.bfe.energieforschung/gpkg/2056/ch.bfe.energieforschung.zip',
  '-'],
 ['Projekte der Koordinationsstelle für nachhaltige Mobilität KOMO',
  'resources',
  'download_url',
  'ff6a5058-7326-4795-9fb3-e5ceb4331801',
  'http://data.geo.admin.ch/ch.bfe.komo-projekte/csv/2056/ch.bfe.komo-projekte.zip',
  'https://data.geo.admin.ch/ch.bfe.komo-projekte/csv/2056/ch.bfe.komo-projekte.zip'],
 ['Projekte der Koordinationsstelle für nachhaltige Mobilität KOMO',
  'resources',
  'url',
  'ff6a5058-7326-4795-9fb3-e5ceb4331801',
  'http://data.geo.admin.ch/ch.bfe.komo-projekte/csv/205

In [ ]:
f = pd.DataFrame(completeList)
f.columns=['title', 'type', 'attribute', 'id', 'old_link', 'new_link']
f

,title,type,attribute,id,old_link,new_link
0,Aktuelle Pilot- und Demonstrationsprojekte des...,resources,download_url,ff6a28fe-f691-4ee1-87df-50ef4d8decf6,https://data.geo.admin.ch/ch.bfe.energieforsch...,-
1,Aktuelle Pilot- und Demonstrationsprojekte des...,resources,url,ff6a28fe-f691-4ee1-87df-50ef4d8decf6,https://data.geo.admin.ch/ch.bfe.energieforsch...,-
2,Projekte der Koordinationsstelle für nachhalti...,resources,download_url,ff6a5058-7326-4795-9fb3-e5ceb4331801,http://data.geo.admin.ch/ch.bfe.komo-projekte/...,https://data.geo.admin.ch/ch.bfe.komo-projekte...
3,Projekte der Koordinationsstelle für nachhalti...,resources,url,ff6a5058-7326-4795-9fb3-e5ceb4331801,http://data.geo.admin.ch/ch.bfe.komo-projekte/...,https://data.geo.admin.ch/ch.bfe.komo-projekte...
4,Projekte der Koordinationsstelle für nachhalti...,resources,download_url,3dd6349f-6e18-4c2f-9e17-83015fe3eb05,http://data.geo.admin.ch/ch.bfe.komo-projekte/...,https://data.geo.admin.ch/ch.bfe.komo-projekte...
...,...,...,...,...,...,...
85,Kennzahlen öffentliche Ladeinfrastruktur Elekt...,resources,description,8110b1b3-1836-4913-834e-864cb00c1656,https://w3c.github.io/csvw/primer/'},-
86,Kennzahlen öffentliche Ladeinfrastruktur Elekt...,resources,url,8110b1b3-1836-4913-834e-864cb00c1656,http://www.uvek-gis.admin.ch/BFE/storymaps/MO_...,-
87,Eignung von Hausdächern für die Nutzung von So...,resources,download_url,6f4342e7-2227-4631-b089-306585393354,http://data.geo.admin.ch/ch.bfe.solarenergie-e...,-
88,Eignung von Hausdächern für die Nutzung von So...,resources,url,6f4342e7-2227-4631-b089-306585393354,http://data.geo.admin.ch/ch.bfe.solarenergie-e...,-


In [ ]:
f.to_csv('resources.csv', index=False)

In [ ]:
df1 = pd.read_csv('urlAndRelations.csv')
df2 = f.copy()

df = pd.concat([df1, df2], ignore_index=True)
df

--------------------------------------------------------------------------------------------------------------

Removing working links which have a strange element in the end

In [2]:
df = pd.read_csv('alles.csv')
df

,title,type,attribute,id,old_link,new_link
0,Kennzahlen öffentliche Ladeinfrastruktur Elekt...,resources,description,8110b1b3-1836-4913-834e-864cb00c1656,"https://w3c.github.io/csvw/primer/',",-
1,Kennzahlen öffentliche Ladeinfrastruktur Elekt...,resources,description,8110b1b3-1836-4913-834e-864cb00c1656,"https://w3c.github.io/csvw/primer/',",-
2,Kennzahlen öffentliche Ladeinfrastruktur Elekt...,resources,description,8110b1b3-1836-4913-834e-864cb00c1656,https://w3c.github.io/csvw/primer/'},-
3,Kennzahlen öffentliche Ladeinfrastruktur Elekt...,resources,description,8110b1b3-1836-4913-834e-864cb00c1656,"https://w3c.github.io/csvw/primer/',",-
4,Aktuelle Pilot- und Demonstrationsprojekte des...,resources,download_url,ff6a28fe-f691-4ee1-87df-50ef4d8decf6,https://data.geo.admin.ch/ch.bfe.energieforsch...,-
...,...,...,...,...,...,...
126,Windatlas Schweiz: Jahresmittel der modelliert...,resources,url,f0904fec-5bb3-4189-bf2f-4d7bd4a560ab,https://data.geo.admin.ch/ch.bfe.windenergie-g...,-
127,Windatlas Schweiz: Jahresmittel der modelliert...,resources,url,eed9fc43-e99c-46a0-8d5f-9ff1dce80d60,https://data.geo.admin.ch/ch.bfe.windenergie-g...,-
128,Windatlas Schweiz: Jahresmittel der modelliert...,resources,url,e149d935-fa29-411e-a7f4-86f411a6ac06,https://data.geo.admin.ch/ch.bfe.windenergie-g...,-
129,Windatlas Schweiz: Jahresmittel der modelliert...,resources,url,7ba07811-4266-45dc-8e33-b98039606e17,https://data.geo.admin.ch/ch.bfe.windenergie-g...,-


In [3]:
df.sort_values(by=['attribute', 'type', 'title'], inplace=True, ignore_index=True)

In [4]:
notWorking = df[df.new_link == '-'].copy()

In [12]:
import re

txt = "sdfkjs.dfk"
x = re.findall("\W", txt)
x

['.']

In [14]:
endC = []

for i in notWorking['old_link']:
  if len(re.findall("\W", i[-1])) != 0 and i[-1] != '/':
    print(i)

https://w3c.github.io/csvw/primer/',
https://w3c.github.io/csvw/primer/',
https://w3c.github.io/csvw/primer/'}
https://w3c.github.io/csvw/primer/',
